Read Json Batch data

In [0]:
dbutils.fs.rm(
    "/Volumes/spark_streaming/files/vol/jsonSource/",
    recurse=True
)

dbutils.fs.rm(
    "/Volumes/spark_streaming/files/vol/jsonSink/",
    recurse=True
)

 
dbutils.fs.mkdirs("/Volumes/spark_streaming/files/vol/jsonSink/data/")

dbutils.fs.mkdirs("/Volumes/spark_streaming/files/vol/jsonSink/checkpoint/")
dbutils.fs.mkdirs("/Volumes/spark_streaming/files/vol/jsonSource/")
 

In [0]:
from pyspark.sql.functions import *

In [0]:
df = spark.read.format('json').option('multiline',True).load('/Volumes/spark_streaming/files/vol/jsonSource/')
display(df)

In [0]:
df = df.withColumn("items",explode_outer("items")).withColumn("metadata",explode_outer("metadata"))
display(df)

In [0]:
df.select("items.item_id","items.price","items.product_name","items.quantity","customer.customer_id","customer.email","customer.name" , "customer.address.city","customer.address.country","customer.address.postal_code","metadata.key","metadata.value","order_id","payment.method","payment.transaction_id","timestamp").display()


In [0]:
df.printSchema()

Read Stream DATA

In [0]:
from pyspark.sql.types import *

In [0]:

myschema = StructType([
    StructField("items", ArrayType(StructType([
        StructField("item_id", StringType(), True),
        StructField("price", DoubleType(), True),
        StructField("product_name", StringType(), True),
        StructField("quantity", IntegerType(), True)
    ]))),
    StructField("customer", StructType([
        StructField("customer_id", IntegerType(), True),
        StructField("email", StringType(), True),
        StructField("name", StringType(), True),
        StructField("address", StructType([
            StructField("city", StringType(), True),
            StructField("country", StringType(), True),
            StructField("postal_code", StringType(), True)
        ]))
    ])),
    StructField("metadata", ArrayType(StructType([
        StructField("key", StringType(), True),
        StructField("value", StringType(), True)
    ]))),
    StructField("payment", StructType([
        StructField("method", StringType(), True),
        StructField("transaction_id", StringType(), True)
    ])),
    StructField("timestamp", StringType(), True),
    StructField("order_id", StringType(), True)
])

In [0]:
df2= spark.readStream.format("json")\
    .option("multiline",True)\
    .schema(myschema)\
    .option("cleanSource","archive")\
    .option("sourceArchiveDir","/Volumes/spark_streaming/files/vol/jsonSink/archive")\
    .load("/Volumes/spark_streaming/files/vol/jsonSource")

df2 = df2\
    .withColumn("items",explode_outer("items"))\
    .withColumn("metadata",explode_outer("metadata"))

# Transformation 
df2 = df2.select("items.item_id","items.price","items.product_name","items.quantity","customer.customer_id","customer.email","customer.name" , "customer.address.city","customer.address.country","customer.address.postal_code","metadata.key","metadata.value","order_id","payment.method","payment.transaction_id","timestamp")

df2.writeStream.format("delta") \
    .outputMode("append") \
    .trigger(once=True) \
    .option("path", "/Volumes/spark_streaming/files/vol/jsonSink/data") \
    .option("checkpointLocation", "/Volumes/spark_streaming/files/vol/jsonSink/checkpoint") \
    .start()


In [0]:
spark.sql('select *from delta.`/Volumes/spark_streaming/files/vol/jsonSink/data`').display()

In [0]:
# dbutils.fs.mkdirs("/Volumes/spark_streaming/files/vol/jsonSink/checkpoint/")

In [0]:
print("hri branch")

In [0]:
print("hri branch test 2")

In [0]:
print("ritesh spark git")